In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from vit_keras import vit

# Define the CNN component (using EfficientNetB0 as an example)
def create_cnn_model(input_shape):
    cnn_base = EfficientNetB0(include_top=False, input_shape=input_shape, pooling='avg')
    cnn_output = cnn_base.output
    return models.Model(inputs=cnn_base.input, outputs=cnn_output)

# Define the ViT component
def create_vit_model(input_shape, num_classes):
    vit_model = vit.vit_b16(
        image_size=input_shape[0],
        activation='softmax',
        pretrained=True,
        include_top=True,
        pretrained_top=False,
        classes=num_classes
    )
    return vit_model

# Combine CNN and ViT models
def create_hybrid_model(input_shape, num_classes):
    # Input layer
    inputs = layers.Input(shape=input_shape)

    # CNN component
    cnn_model = create_cnn_model(input_shape)
    cnn_features = cnn_model(inputs)

    # ViT component
    vit_model = create_vit_model(input_shape, num_classes)
    vit_features = vit_model(inputs)

    # Concatenate CNN and ViT features
    combined_features = layers.Concatenate()([cnn_features, vit_features])

    # Fully connected layers
    x = layers.Dense(512, activation='relu')(combined_features)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create and compile model
    hybrid_model = models.Model(inputs=inputs, outputs=outputs)
    hybrid_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return hybrid_model

# Define input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 2

# Create the hybrid model
model = create_hybrid_model(input_shape, num_classes)

# Display the model summary
model.summary()
